### FLAG PARAMETER

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window

**Create a flag whether it is first run or Incremental Run**

_Set Incremental parameter as 0_

In [0]:
dbutils.widgets.text('Incremental_flag','0')

**To get the incremental Flag value**

In [0]:
incremental_flag = dbutils.widgets.get('Incremental_flag')
print(incremental_flag)
print(type(incremental_flag))

#### CREATING DIMENSION MODEL

In [0]:
%sql
select* from parquet.`abfss://silver@azureproject8940.dfs.core.windows.net/carsales`


**Fetch the moden Dim columns - Model-id and category and ModelID should be distinct/Unique**

In [0]:
%sql
select distinct
  Dealer_ID as Dealer_ID,
  DealerName as Dealer_Name
from
  parquet.`abfss://silver@azureproject8940.dfs.core.windows.net/carsales`

**Store the data in Dataframe**

In [0]:
df_src=spark.sql('''
    select distinct
    Dealer_ID as Dealer_ID,
  DealerName as Dealer_Name
from
  parquet.`abfss://silver@azureproject8940.dfs.core.windows.net/carsales`
''')

In [0]:
display(df_src)

### Dim_Model_Sink - initital & Incremental

#### Create Schema - Schema
`Where 1=0` is means the condition is false and always bring only schema and not data

**Now apply the condition- if the firsttime load (bring schema) if table and data available bring the value**

In [0]:
# If table already available then we can directly read the table
if spark.catalog.tableExists("`car-sales-catalg`.gold.dim_dealer"):
    # When the actual data brings from table we dnt need to mention where 1=0 and 1 as Dim_key
    df_sink = spark.sql(
        """
    select Dim_Dealer_Key, Dealer_ID, Dealer_Name
    from `car-sales-catalg`.gold.dim_dealer
    """
    )
# If we dont have the table then we need to create dummy value as 1 as Dim_key and add where 1=0
else:
    df_sink = spark.sql(
        """
    select 1 as Dim_Dealer_Key, Dealer_ID, DealerName as Dealer_Name
    from parquet.`abfss://silver@azureproject8940.dfs.core.windows.net/carsales`
    where 1=0
    """
    )

In [0]:
df_sink.display()

### Filtering Old and New records

In [0]:
df_filter = df_src.join(
    df_sink, df_src["Dealer_ID"] == df_sink["Dealer_ID"], "left"
).select(df_src["Dealer_ID"], df_src["Dealer_Name"], df_sink["Dim_Dealer_Key"])

display(df_filter)

Based on the condition-if dimkey null it is consider new record - need to generate `Surogate Key`. If not null this is consider as old record and it will be updated.

Also we will consider the surrogate key - starts from 0 and max value and use the incremental Flag - in below section

**DF_FILTER_OLD**

In [0]:
df_filter_old = df_filter.filter(df_filter['Dim_Dealer_Key'].isNotNull())

In [0]:
display(df_filter_old)

**DF_FILTER_NEW**

**Dont required Surrogate key column as it has no value.**

In [0]:
df_filter_new = df_filter.filter(df_filter['Dim_Dealer_Key'].isNull()).select(df_filter['Dealer_ID'],df_filter['Dealer_Name'])
display(df_filter_new)

### Create Surrogate Key

**Fetch the max surrogate key**

In [0]:
if incremental_flag == "0":
    max_value = 0

else:
    max_value_df = spark.sql(
        "select max(Dim_Dealer_Key) from `car-sales-catalg`.gold.dim_dealer"
    )
    max_value = max_value_df.collect()[0][0]

**Create Surrogate key column & add max Surrogate key value**

In [0]:
df_filter_new=df_filter_new.withColumn('Dim_Dealer_Key',max_value+monotonically_increasing_id()+1)

In [0]:
display(df_filter_new)

**Create Final DF - DF Filterold + DF Filter New**

In [0]:
df_final=df_filter_old.union(df_filter_new)
display(df_final)

### SCD - Slowly Changing Dimension Type- 1 (Upsert)

**Upsert = Update+Insert**

In [0]:
from delta.tables import DeltaTable 

In [0]:
# Incremental
if spark.catalog.tableExists('`car-sales-catalg`.gold.dim_dealer'):
    # create the deltatable obj
    delta_tbl = DeltaTable.forPath(
        spark, "abfss://gold@azureproject8940.dfs.core.windows.net/dim_dealer"
    )
    # upsert concept - Merge
    delta_tbl.alias("trg").merge(
        df_final.alias("src"), "trg.Dim_Dealer_Key == src.Dim_Dealer_Key"
        #when ID matches- Updates and If not matches insert
    ).whenMatchedUpdateAll().whenNotMatchedInsertAll().execute()

# Initial
else:
    df_final.write.format("delta").mode("overwrite").option(
        "path", "abfss://gold@azureproject8940.dfs.core.windows.net/dim_dealer"
    ).saveAsTable('`car-sales-catalg`.gold.dim_dealer')

In [0]:
%sql
select * from `car-sales-catalg`.gold.dim_dealer